# JamDict Playgound 0x05

This notebook is for collecting all Kanji for JLPT N5 and JLPT N4 and store them accordingly. Since we already have a database of all kanji Stroke data, which contains all kanji, we can use this as a reference for retrieving the Kanji from the databases accessible by jamdict.

In [ ]:
import json
import os

from jamdict import Jamdict

In [ ]:
jamdict = Jamdict()

## Load the Strokedata

In [ ]:
DATA_DIR = '../data'
RADICAL_STROKE_DATA = 'kanjiStrokeData.json'

with open(os.path.join(DATA_DIR,RADICAL_STROKE_DATA),'r') as outputFile:
    all_kanji = json.load( outputFile);
    



This dataset only contains the  old JLPT data. But what happened is that JLPT-N5 is the old JLPT-4 and JLPT-N4 is the old JLPT-3 level. The Old JLPT-2 was split into the new JLPT-N2 and JLPT-N3 classes to make the JLPT-2 more achievable. So We can generate the data for JLPT N4 and JLPT-N5 and probably JLPT-N1 only JLPT-N2 and N3 needs some special attention.

In [ ]:
jlpt = {}

current_kanji = 0
for kanji in all_kanji['list']:
    current_kanji+=1
    if(current_kanji%200 == 0):
        print("iteration: {}".format(current_kanji))
    result = jamdict.lookup(kanji,strict_lookup=True)
    
    for char in result.chars:
        # print(dir(char))
        if char.jlpt and (not char.jlpt in jlpt.keys()):
            print("unknown jlpt: {} found.".format(char.jlpt))
            jlpt[char.jlpt] = list()
            
        if( char.jlpt ):
            jlpt[char.jlpt].append(char)

    # break;

In [ ]:
jlpt

##  Function to save a dictionary

In [ ]:
def get_readings(rm):
    readings = rm['readings']
    # extract
    # type="ja_kun" / value
    kun = [ reading['value'] for reading in readings if reading['type']=='ja_kun']
    #print("kun ")
    #print(kun)
    
    # type="ja_on" / value
    on = [ reading['value'] for reading in readings if reading['type']=='ja_on']
    #print("on ")
    #print(on)
    
    return kun, on

def get_meanings_eng(rm):
    meanings = rm['meanings']
    # extract
    # m_lang=""/value
    eng_meanings = [ meaning['value'] for meaning in meanings if meaning['m_lang']=='']
    #print('meanings')
    #print(eng_meanings)
    return eng_meanings

def get_reference_id(allrefs, dict_name):
    # print(allrefs)
    for ref in allrefs:
        if ref['type']==dict_name :
            return int(ref['value'])
    return 10000

def save_dictionary(char_list, list_name:str):
    myDict = {
        'name':list_name, 
        'length':len(char_list),
        'list':[]}
    
    reference={}
    
    for char in char_list:
        myValue = { 'kanji': char.literal }
        #print(dir(char))
        
        char_jsondata = char.to_json()
        # print(char_jsondata.keys())
        
        rm=char_jsondata['rm']
        #print("RM(0)")
        #print(rm[0])
        kun,on = get_readings(rm[0])
        myValue['kana'] = {'kun':kun, 'on':on }
        myValue['meaning'] = get_meanings_eng(rm[0])
        
        myDict['list'].append(myValue)
        # 'nelson_c', 'nelson_n' 'halpern_njecd', 'halpern_kkd', 'halpern_kkld', 'halpern_kkld_2ed', 'heisig', 'heisig6',
        # 'gakken', 'oneill_names', 'oneill_kk', 'moro', 'henshall', 'sh_kk', 'sh_kk2', 'sakade', 'jf_cards', 'henshall3', 
        # 'tutt_cards', 'crowley', 'kanji_in_context', 'kodansha_compact', 'maniette'
        reference[char.literal] = get_reference_id(char_jsondata['dic_refs'],'heisig6')
        # break;
   
    mydict_list = myDict['list']
    sorted_mydict_list = sorted(mydict_list, key=lambda entry: reference[entry['kanji']])
    myDict['list'] = sorted_mydict_list
    
    with open(os.path.join(DATA_DIR, "quiz_data", list_name+".json"),'w') as jsonOutputFile:
        json.dump(myDict,jsonOutputFile,indent=2)
        pass

In [ ]:
save_dictionary(jlpt['4'], 'JLPT N5 Kanji')
len(jlpt['4'])

In [ ]:
save_dictionary(jlpt['3'], 'JLPT N4 Kanji')
len(jlpt['3'])

In [ ]:
# We have to split these Kanji into two categories ... N3(420) and N2(319)  Old JLPT 2 (739 Kanji)

#save_dictionary(jlpt['2'], 'JLPT N3 Kanji')
#save_dictionary(jlpt['2'], 'JLPT N2 Kanji')
len(jlpt['2'])

In [ ]:
#save_dictionary(jlpt['1'], 'JLPT N1 Kanji')
# The length is longer than expected... 1207 instead of expected 1011 Kanji
len(jlpt['1'])